# Tutorial

This notebook is split into two parts:
-  Part 1: Making basis vectors
-   Part 2: Projecting onto basis vectors


Those who are only interested in using this package for octahedron environment could safely skip Part 1 and start from Part 2.

# Part 1: Making basis vectors

Coordinates of the ideal structure must be set manually. Order and axes does not matter.

In [1]:
from basis_generator import * # change accordingly

# constants
ideal_coords = [
    [-1,  0,  0],
    [0, -1,  0],
    [0,  0, -1],
    [0,  0,  1],
    [0,  1,  0],
    [1,  0,  0]
]

Numerical algorithm (as introduced in the paper) is implemented to work for any symmetry environment.
Some tuning may be required to minimise numerical errors.
Irrep for translation and rotation must be set manually.

In [2]:
with open('basis_example_numerical.json', 'w') as f:
    f.write(json.dumps(
        basis_generating_machine_character(
            ideal_coords, point_group='m-3m',
            basis_sorter=lambda x: sort_basis_numerical(
                dict_basis=x, translation_irrep='T1u', 
                rotation_irrep='T1g', ideal_coords=ideal_coords))
    ) + '\n')


For octahedron, analytical solution is provided.
For simpler point groups, one may prefer to implement analytically.

In [3]:
with open('basis_example_analytical.json', 'w') as f:
    f.write(json.dumps(
        basis_generating_machine_character(
            ideal_coords, point_group='m-3m',
            basis_sorter=sort_basis_analytical_octahderon)
    ) + '\n')


We can see that the specific values of the basis set differ, as mentioned in the paper.

In [4]:
!diff -q basis_example_numerical.json basis_example_analytical.json

Files basis_example_numerical.json and basis_example_analytical.json differ


# Part 2: Projecting onto basis vectors

In [5]:
# to disable CrystalNN warnings
import warnings
warnings.filterwarnings("ignore")

This part is independent and does not require execution of Part 1.

In [6]:
## Prepare
from polyhedron_analysis import * # change accordingly
import pymatgen.core.structure

# convert cif to pymatgen structure
mp_struct = pymatgen.core.structure.Structure.from_file('example_structure/BaTiO3_mp-558125_computed.cif')
centre_atom = 7

## The structure

We will look into the edge octahedron of the 1D TiO6 chain in hexagonal phase BaTiO3 (site 7).

![BaTiO3](example_structure/BaTiO3.png)

## simple usage

In [7]:
# main analysis
print('#Eg, T2g, T1u, T2u, T1u(centre)')
print(calc_distortions_from_struct_octahedron_withcentre(mp_struct, centre_atom))

#Eg, T2g, T1u, T2u, T1u(centre)
[0.01545027 0.06961483 0.11746016 0.01525142 0.11227169]


## "Pro" usage

In this part, we compare analytically and numerically calculated basis and show that the final result are the same.

In [8]:
# constants
ideal_coords = [
    [-1,  0,  0],
    [0, -1,  0],
    [0,  0, -1],
    [0,  0,  1],
    [0,  1,  0],
    [1,  0,  0]
]

The detection of nearest neighbours relys on CrystalNN.
We find that the result is not so sensitive for ABO3, but may require tuning for more subtle structures.

In [9]:
# common process

## handle nearest neighbours
mp_struct.get_neighbor_list(r=3.5)
nearest_neighbour_finder = pymatgen.analysis.local_env.CrystalNN()
temp_dict = sorted(
    nearest_neighbour_finder.get_nn_info(
        structure=mp_struct, n=centre_atom), key=lambda x: -x['weight']
)[:len(ideal_coords)]

## numerically calculated basis sets

In [10]:
# read json
with open('basis/octahedron_basis.json', 'r') as f:
    dict_basis = json.load(f)
irrep_distortions = []
for irrep in dict_basis.keys():
    for elem in dict_basis[irrep]:
        irrep_distortions.append(elem)

# define "molecules"
temp_coords = np.array([
    mp_struct[d['site_index']].coords
    + mp_struct.lattice.get_cartesian_coords(d['image'])
    for d in temp_dict
])
molecule_origin = np.mean(temp_coords, axis=0)
ave_bond = temp_coords - molecule_origin
ave_bond = np.mean(np.sqrt(np.sum(ave_bond * ave_bond, axis=1)))
pymatgen_molecule = construct_molecule(
    struct=mp_struct,
    centre_atom=centre_atom,
    nearest_neighbour_indices=[d['site_index'] for d in temp_dict],
    ave_bond=ave_bond,
    images=[d['image'] for d in temp_dict],
    origin=molecule_origin,
)
pymatgen_molecule_ideal = construct_molecule_ideal(
    ideal_coords, pymatgen_molecule.species)

# transform
(pymatgen_molecule, matrix_rotation, _) = match_molecules(
    pymatgen_molecule, pymatgen_molecule_ideal)

# project
distortion_amplitudes = calc_displacement(
    pymatgen_molecule, pymatgen_molecule_ideal, irrep_distortions
)

# calc projection for central atom
centre_atom_amplitude = calc_displacement_centre(
    mp_struct, centre_atom, molecule_origin, ave_bond, matrix_rotation)
centre_atom_amplitude = np.sqrt(
    np.sum(centre_atom_amplitude * centre_atom_amplitude))

# average
distortion_amplitudes = distortion_amplitudes * distortion_amplitudes
temp_list = []
count = 0
for irrep in dict_basis:
    dim = len(dict_basis[irrep])
    temp_list.append(np.sum(distortion_amplitudes[count:count + dim]))
    count += dim
distortion_amplitudes = np.sqrt(temp_list)[3:]

print('#Eg, T2g, T1u, T2u, T1u(centre)')
print(np.concatenate((distortion_amplitudes, [centre_atom_amplitude])))

#Eg, T2g, T1u, T2u, T1u(centre)
[0.01545027 0.06961483 0.11746016 0.01525142 0.11227169]


## analytically calculated basis sets

In [11]:
# read json
with open('basis/octahedron_basis_analytical.json', 'r') as f:
    dict_basis = json.load(f)
irrep_distortions = []
for irrep in dict_basis.keys():
    for elem in dict_basis[irrep]:
        irrep_distortions.append(elem)

# define "molecules"
temp_coords = np.array([
    mp_struct[d['site_index']].coords
    + mp_struct.lattice.get_cartesian_coords(d['image'])
    for d in temp_dict
])
molecule_origin = np.mean(temp_coords, axis=0)
ave_bond = temp_coords - molecule_origin
ave_bond = np.mean(np.sqrt(np.sum(ave_bond * ave_bond, axis=1)))
pymatgen_molecule = construct_molecule(
    struct=mp_struct,
    centre_atom=centre_atom,
    nearest_neighbour_indices=[d['site_index'] for d in temp_dict],
    ave_bond=ave_bond,
    images=[d['image'] for d in temp_dict],
    origin=molecule_origin,
)
pymatgen_molecule_ideal = construct_molecule_ideal(
    ideal_coords, pymatgen_molecule.species)

# transform
(pymatgen_molecule, matrix_rotation, _) = match_molecules(
    pymatgen_molecule, pymatgen_molecule_ideal)

# project
distortion_amplitudes = calc_displacement(
    pymatgen_molecule, pymatgen_molecule_ideal, irrep_distortions
)

# calc projection for central atom
centre_atom_amplitude = calc_displacement_centre(
    mp_struct, centre_atom, molecule_origin, ave_bond, matrix_rotation)
centre_atom_amplitude = np.sqrt(
    np.sum(centre_atom_amplitude * centre_atom_amplitude))

# average
distortion_amplitudes = distortion_amplitudes * distortion_amplitudes
temp_list = []
count = 0
for irrep in dict_basis:
    dim = len(dict_basis[irrep])
    temp_list.append(np.sum(distortion_amplitudes[count:count + dim]))
    count += dim
distortion_amplitudes = np.sqrt(temp_list)[3:]

print('#Eg, T2g, T1u, T2u, T1u(centre)')
print(np.concatenate((distortion_amplitudes, [centre_atom_amplitude])))

#Eg, T2g, T1u, T2u, T1u(centre)
[0.01545027 0.06961483 0.11746016 0.01525142 0.11227169]


Within the range of numerical error, the results should show
```
#Eg, T2g, T1u, T2u, T1u(centre)
[0.01545027 0.06961483 0.11746016 0.01525142 0.11227169]
```
in both cases.